In [ ]:
import pandas as pd
import os
import spacy
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from collections import Counter

In [ ]:
#import yaml

#with open("../config.yaml", "r") as file:
#    config = yaml.safe_load(file)

#pd.read_csv(config["input_data"]["file2"])
#df.to_csv(config["output_data"]["file2"])

In [ ]:

nlp = spacy.load("en_core_web_sm")


## Clean column name by lower and replace white space

In [ ]:
# sentiment_df
# co_ai_adop_df
# occu_gro_df
# occ_w_df

In [ ]:
#REDDIT COMMENTS AKA ai job sentiment
file_p= '../data/raw/reddit_comments_combined.csv'
sentiment_df= pd.read_csv(file_p)
sentiment_df.head()

In [ ]:
sentiment_df.shape

In [ ]:
sentiment_df.columns = sentiment_df.columns.str.replace(' ', '_').str.lower()
sentiment_df = sentiment_df.replace(r'\s+', '_', regex=True)

In [ ]:
# View nulls  Sentiment
print("Missing values per column:\n", sentiment_df.isnull().sum())

In [ ]:
 for col in sentiment_df.columns:
    if sentiment_df[col].isnull().sum() == 0:
        continue  # Skip columns with no missing values
    
    if sentiment_df[col].dtype == 'object':
        mode_val = sentiment_df[col].mode()
        if not mode_val.empty:
            sentiment_df[col] = sentiment_df[col].fillna(mode_val[0])
        else:
            sentiment_df[col] = sentiment_df[col].fillna("unknown")  # fallback
    else:
        mean_val = sentiment_df[col].mean()
        sentiment_df[col] = sentiment_df[col].fillna(mean_val)

In [ ]:
#fill nan values of sentiment
sentiment_df['comment_author'] = sentiment_df['comment_author'].fillna('anonymous')


In [ ]:
# Remove duplicates
sentiment_df = sentiment_df.drop_duplicates().reset_index(drop=True)

In [ ]:
print("Remaining nulls:\n", sentiment_df.isnull().sum())


In [ ]:
#  Drop duplicates and reset index
sentiment_df = sentiment_df.drop_duplicates().reset_index(drop=True)


In [ ]:
sentiment_df = sentiment_df.drop(['post_id', 'comment_created'], axis=1)


In [ ]:
sentiment_df.columns

In [ ]:
sentiment_df.dtypes

In [ ]:
print("Final shape:", sentiment_df.shape)

In [ ]:
sentiment_df['comment_body'] = sentiment_df['comment_body'].astype(str)

In [ ]:
nlp = spacy.load("en_core_web_sm")


In [ ]:
# sentiment words list
positive_words = {
    'safe', 'secure', 'confident', 'optimistic', 'opportunity', 'creative', 
    'valuable', 'promising', 'empowered', 'supportive', 'hopeful', 'excited','good', 'great', 
    'helpful', 'smart', 'efficient', 'love', 'amazing', 'brilliant'
}

negative_words = {
    'worried', 'afraid', 'fear', 'replace', 'jobless', 'redundant', 
    'useless', 'anxious', 'threatened', 'concerned', 'scared', 'depressed',
    'bad', 'terrible', 'useless', 'dumb', 'hate', 'annoying', 'slow', 'stupid'
}


In [ ]:
def analyze_sentiment_spacy(comment):
    doc = nlp(comment.lower())
    pos_count = sum(token.text in positive_words for token in doc)
    neg_count = sum(token.text in negative_words for token in doc)
    
    if pos_count > neg_count:
        return 'positive'
    elif neg_count > pos_count:
        return 'negative'
    else:
        return 'neutral'

sentiment_df['sentiment'] = sentiment_df['comment_body'].apply(analyze_sentiment_spacy)


In [ ]:
print(sentiment_df['sentiment'].value_counts())


In [ ]:
# Combine all comments into one string
all_text = ' '.join(sentiment_df['comment_body'].dropna().tolist()).lower()

# Tokenize with spaCy and filter for actual words (no punctuation/numbers/stop words)
#Create a list of "clean" words:
#    - is_alpha: only keep words (no numbers or punctuation)
#    - not token.is_stop: remove common words like "the", "and", "is", etc.
#    - token.lemma_: get the base form of each word (e.g., "working" → "work")
doc = nlp(all_text)
tokens = [token.lemma_ for token in doc if token.is_alpha and not token.is_stop]

# Count the top words
word_freq = Counter(tokens).most_common(300)

# Generate wordcloud
wordcloud = WordCloud(width=800, height=400, background_color='white').generate_from_frequencies(dict(word_freq))

# Plot
plt.figure(figsize=(12, 6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Top 300 Words in Reddit AI Job Comments', fontsize=16)
# Displaying the plot
plt.savefig("../slides/visuals/Top 300 Words in Reddit AI Job Comments.png", transparent=True, format="png")
plt.show()

## CLEANING OF EMPLOYMENT PROJECTIONS

In [ ]:
import pandas as pd

In [ ]:
# Employment_projections = pd.read_csv(r'..\data\raw\Employment_projections.csv')



In [ ]:
# # 1. Clean column names
# Employment_projections.columns = Employment_projections.columns.str.lower().str.replace(" ", "_").str.strip()

In [ ]:
# 2. View nulls
# print("Missing values per column:\n", Employment_projections.isnull().sum())

In [ ]:
# print(df.columns.tolist())



In [ ]:
# df.columns = df.columns.str.lower().str.replace(" ", "_").str.replace(",", "").str.strip()
# print(df.columns.tolist())


In [ ]:
# df['work_experience_in_a_related_occupation'] = df['work_experience_in_a_related_occupation'].fillna('Not Required')


In [ ]:
#df['typical_on-the-job_training'] = df['typical_on-the-job_training'].fillna('None')
 

In [ ]:
# Employment_projections.to_csv(r'..\data\clean\Employment_projection.csv', index=False, encoding='utf-8', sep=";")

## CLEANING OF OCCUPATION GROWTH

In [ ]:
#print(df[['work_experience_in_a_related_occupation', 'typical_on-the-job_training']].isnull().sum())


In [ ]:
#READ data occupation growth
import os
folder_path = '../data/raw/'
matrix_files = [
    "National_Employment_Matrix_IND_31-330.csv",
    "National_Employment_Matrix_IND_230000.csv",
    "National_Employment_Matrix_IND_541200.csv",
    "National_Employment_Matrix_IND_541300.csv",
    "National_Employment_Matrix_IND_610000.csv",
    "National_Employment_Matrix_IND_710000.csv",
    "National_Employment_Matrix_IND_721000.csv",
    "National_Employment_Matrix_OCC_39-3011.csv"
]

# Load and combine all matrix datasets
matrix_dfs = []
for file in matrix_files:
    #path = os.path.join(folder_path, file)
    occu_gro_df = pd.read_csv(folder_path + file, on_bad_lines='skip')  # Skip broken lines
    occu_gro_df['source_file'] = file  # Track origin
    matrix_dfs.append(occu_gro_df)

# Merge into one dataframe
occu_gro_df = pd.concat(matrix_dfs, ignore_index=True)
print("Employment matrix shape:", occu_gro_df.shape)
occu_gro_df.head(5)

# occu_gro_df= pd.read_csv(r'..\data\clean\occupation_growth.csv')
# occu_gro_df.head()

In [ ]:
# 2. Clean column names
occu_gro_df.columns = (
    occu_gro_df.columns
    .str.lower()
    .str.replace(" ", "_")
    .str.replace(",", "")
    .str.strip()
)

In [ ]:
print(occu_gro_df.columns.tolist())


In [ ]:
# Drop duplicates and reset index
occu_gro_df = occu_gro_df.drop_duplicates().reset_index(drop=True)

In [ ]:
occu_gro_df.columns


In [ ]:
occu_gro_df = occu_gro_df.loc[:, ['occupation_title', 'occupation_code', '2023_employment', 'projected_2033_employment','employment_percent_change_2023-2033','employment_change_2023-2033']]

In [ ]:
occu_gro_df.head(10)

In [ ]:
occu_gro_df.dtypes

In [ ]:
# Remove commas and convert to float
occu_gro_df['2023_employment'] = occu_gro_df['2023_employment'].str.replace(',', '').astype(float)
occu_gro_df['projected_2033_employment'] = occu_gro_df['projected_2033_employment'].str.replace(',', '').astype(float)

occu_gro_df['occupation_code'].astype(str)

## CLEANING  OF OCCUPATION WAGE

In [ ]:
#READ data occupation wages

import csv

# Reload the re-uploaded file after reset
file_path = '../data/raw/all_data_M_2024.csv'

# Attempt to load while skipping problematic lines
try:
    occ_w_df = pd.read_csv(
        file_path,
        quoting=csv.QUOTE_MINIMAL,
        encoding='utf-8',
        on_bad_lines='skip'  # skips problematic lines
    )
    cleaned_preview = occ_w_df.head()
except Exception as e:
    cleaned_preview = f"Error during loading: {e}"

cleaned_preview
# Reload the file using semicolon delimiter
occ_w_df = pd.read_csv(file_path, delimiter=';', encoding='utf-8', on_bad_lines='skip')

# Show the cleaned preview
occ_w_df.head()

In [ ]:
 #2. Clean column names
occ_w_df.columns = (
    occ_w_df.columns
    .str.lower()
    .str.replace(" ", "_")
    .str.replace(",", "")
    .str.strip()
)


In [ ]:
# 3. Print missing values for review
print("Missing values before cleaning:\n", occ_w_df.isnull().sum())


In [ ]:
# Drop duplicate columns (e.g. measure, time_period, etc.)
occ_w_df = occ_w_df.loc[:, ~occ_w_df.columns.duplicated()].reset_index(drop=True)

In [ ]:
# Drop columns with more than 90% missing values
missing_ratio = occ_w_df.isnull().mean()
cols_to_drop = missing_ratio[missing_ratio > 0.9].index
occ_w_df.drop(columns=cols_to_drop, inplace=True)
print(f"Dropped columns with >90% missing values:\n{list(cols_to_drop)}\n")

In [ ]:
occ_w_df.columns

In [ ]:
# Fill missing values
for col in occ_w_df.columns:
    if occ_w_df[col].isnull().sum() == 0:
        continue
    if occ_w_df[col].dtype == 'object':
        mode_val = occ_w_df[col].mode()
        occ_w_df[col] = occ_w_df[col].fillna(mode_val[0] if not mode_val.empty else "unknown")
    else:
        occ_w_df[col] = occ_w_df[col].fillna(occ_w_df[col].mean())

In [ ]:
# Try to convert any remaining object columns to numeric
for col in occ_w_df.columns:
    if occ_w_df[col].dtype == 'object':
        occ_w_df[col] = occ_w_df[col].replace(r'[,\$]', '', regex=True)
        try:
            occ_w_df[col] = pd.to_numeric(occ_w_df[col])
        except:
            pass  # leave as object if not convertible

In [ ]:
print(f"Final shape:", {occ_w_df.shape})

In [ ]:
occ_w_df.head(11)


In [ ]:
occ_w_df=occ_w_df.loc[:, ['tot_emp', 'h_mean', 'a_mean', 'naics_title']]


In [ ]:
occ_w_df = occ_w_df.rename(columns={'naics_title':'sector'})

In [ ]:
occ_w_df['sector'].unique()


In [ ]:
occ_w_df.columns

## CLEANING OF COUNTRY  AI ADOPTION

In [ ]:
file_path = '../data/raw/Oecd.csv'

# Read the Excel file
co_ai_adop_df = pd.read_csv(file_path)

# Display the first few rows
co_ai_adop_df.tail()

In [ ]:
# 2. Clean column names
co_ai_adop_df.columns = (
    co_ai_adop_df.columns
    .str.lower()
    .str.replace(" ", "_")
    .str.replace(",", "")
    .str.replace("@", "")
    .str.replace("+", "")
    .str.strip()
)

In [ ]:
# 3. Check for missing values
print("Missing values before cleaning:\n", co_ai_adop_df.isnull().sum())

In [ ]:
# Drop exact duplicate columns
co_ai_adop_df = co_ai_adop_df.loc[:, ~co_ai_adop_df.columns.duplicated()]



In [ ]:
# Drop columns with > 90% missing values
threshold = 0.9
missing_ratio = co_ai_adop_df.isnull().mean()
columns_to_drop = missing_ratio[missing_ratio > threshold].index
co_ai_adop_df.drop(columns=columns_to_drop, inplace=True)


In [ ]:
print("Columns dropped due to high missing values:\n", list(columns_to_drop))

# Fill remaining missing values
for col in co_ai_adop_df.columns:
    if co_ai_adop_df[col].isnull().sum() == 0:
        continue
    if co_ai_adop_df[col].dtype == 'object':
        mode_val = co_ai_adop_df[col].mode()
        co_ai_adop_df[col] = co_ai_adop_df[col].fillna(mode_val[0] if not mode_val.empty else 'unknown')
    else:
        co_ai_adop_df[col] = co_ai_adop_df[col].fillna(co_ai_adop_df[col].mean())

In [ ]:
f = co_ai_adop_df.drop_duplicates().reset_index(drop=True)


In [ ]:
print(f"Cleaned Oecd dataset saved. Final shape: {co_ai_adop_df.shape}")

## Formatting data types
Dealing with Null values
Identify any columns with null or missing values. Identify how many null values each column has. You can use the isnull() function in pandas to find columns with null values.

Decide on a strategy for handling the null values. There are several options, including:

Drop the rows or columns with null values
Fill the null values with a specific value (such as the column mean or median for numerical variables, and mode for categorical variables)
Fill the null values with the previous or next value in the column

-- Implement your chosen strategy to handle the null values. You can use the fillna() function in pandas to fill null values or dropna() function to drop null values.

Verify that your strategy has successfully handled the null values. You can use the isnull() function again to check if there are still null values in the dataset.

Remember to document your process and explain your reasoning for choosing a particular strategy for handling null values.

After formatting data types, as a last step, convert all the numeric variables to integers.

 ## Dealing with duplicates
Use the .duplicated() method to identify any duplicate rows in the dataframe.

Decide on a strategy for handling the duplicates. Options include:

Dropping all duplicate rows
Keeping only the first occurrence of each duplicated row
Keeping only the last occurrence of each duplicated row
Dropping duplicates based on a subset of columns
Dropping duplicates based on a specific column
Implement your chosen strategy using the drop_duplicates() function.

Verify that your strategy has successfully handled the duplicates by checking for duplicates again using .duplicated().

Remember to document your process and explain your reasoning for choosing a particular strategy for handling duplicates.

Save the cleaned dataset to a new CSV file.

Hint: after dropping duplicates, reset the index to ensure consistency.

## Creating functions on a separate py file
Put all the data cleaning and formatting steps into functions, and create a main function that performs all the cleaning and formatting.

Write these functions in separate .py file(s). By putting these steps into functions, we can make the code more modular and easier to maintain.

Hint: autoreload module is a utility module in Python that allows you to automatically reload modules in the current session when changes are made to the source code. This can be useful in situations where you are actively developing code and want to see the effects of changes you make without having to constantly restart the Python interpreter or Jupyter Notebook kernel.

## Analyzing Clean and Formated Data

In [ ]:
#Save reddit csv


save_path = '../data/clean/'
#os.makedirs(save_path, exist_ok=True)

sentiment_df.to_csv(save_path + 'ai_job_sentiments.csv', index=False, encoding='utf-8', sep=';')
#sentiment_df.to_csv(os.path.join(save_path, 'ai_job_sentiments.csv'), index=False, encoding='utf-8', sep=';')


In [ ]:
#Save occupation_growth csv
occu_gro_df.to_csv(save_path + 'occupation_growth.csv', index=False, encoding='utf-8', sep=";")

In [ ]:
#Save occupation_wages

occ_w_df.to_csv(save_path + 'occupation_wages.csv', index=False, encoding='utf-8', sep=";")

In [ ]:
#Save country_ai_adoption
co_ai_adop_df.to_csv(save_path + 'country_ai_adoption.csv', index=False, encoding='utf-8', sep=";")